In [170]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#

### Import libraries yang digunakan
melakukan import terhadap library yang digunakan

In [171]:
!pip install Sastrawi

In [172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.metrics.pairwise import cosine_similarity

from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
from tensorflow.keras.models import load_model

tv = TfidfVectorizer(max_features=5000)
stem = StemmerFactory().create_stemmer()
stopword = StopWordRemoverFactory().create_stop_word_remover()

### Read file
membaca file pada data sekaligus melakukan cek terhadap kolom dan baris dan juga isi dari file

In [173]:
data_food1 = pd.read_csv('Food Ingredients and Recipe Dataset with Image Name Mapping.csv')

In [174]:
data_food1.head()

,Food_Id,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [175]:
data_user = pd.read_csv('user.csv')

In [7]:
data_user.head()

,User_Id,Location,Age
0,1,"Semarang, Jawa Tengah",20
1,2,"Bekasi, Jawa Barat",21
2,3,"Cirebon, Jawa Barat",23
3,4,"Bekasi, Jawa Barat",21
4,5,"Lampung, Sumatera Selatan",20


In [8]:
data_food_loves = pd.read_csv('food_loves.csv')

In [176]:
data_food_loves.head()

,User_Id,Food_Id,Food_Loves
0,1,179,3
1,1,344,2
2,1,5,5
3,1,373,3
4,1,101,4


### Data Preparation

In [177]:
data_food1.isna().sum()

Food_Id                0
Title                  5
Ingredients            0
Instructions           8
Image_Name             0
Cleaned_Ingredients    0
dtype: int64

In [178]:
data_user.isna().sum()

User_Id     0
Location    0
Age         0
dtype: int64

In [179]:
data_food_loves.isna().sum()

User_Id       0
Food_Id       0
Food_Loves    0
dtype: int64

In [180]:
data_rekomendasi = data_food1
data_rekomendasi


,Food_Id,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."
...,...,...,...,...,...,...
13496,13496,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"['1 cup all-purpose flour', '2/3 cup unsweeten..."
13497,13497,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,"['1 preserved lemon', '1 1/2 pound butternut s..."
13498,13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",rice-with-soy-glazed-bonito-flakes-and-sesame-...,['Leftover katsuo bushi (dried bonito flakes) ...
13499,13499,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,spanakopita-107344,['1 stick (1/2 cup) plus 1 tablespoon unsalted...


#### Modeling with Content Based Filtering

In [181]:
def preprocessing(data):
    data = data.lower()
    data = stem.stem(data)
    data = stopword.remove(data)
    return data

In [183]:
data_content_based_filtering = data_rekomendasi.copy()
data_content_based_filtering['Bahan'] = data_content_based_filtering['Title'] + ' ' + data_content_based_filtering['Cleaned_Ingredients']
data_content_based_filtering

,Food_Id,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bahan
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...",Miso-Butter Roast Chicken With Acorn Squash Pa...
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...",Crispy Salt and Pepper Potatoes ['2 large egg ...
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...",Thanksgiving Mac and Cheese ['1 cup evaporated...
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Italian Sausage and Bread Stuffing ['1 (¾- to ...
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","Newton's Law ['1 teaspoon dark brown sugar', '..."
...,...,...,...,...,...,...,...
13496,13496,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Brownie Pudding Cake ['1 cup all-purpose flour...
13497,13497,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Israeli Couscous with Roasted Butternut Squash...
13498,13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",rice-with-soy-glazed-bonito-flakes-and-sesame-...,['Leftover katsuo bushi (dried bonito flakes) ...,Rice with Soy-Glazed Bonito Flakes and Sesame ...
13499,13499,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,spanakopita-107344,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Spanakopita ['1 stick (1/2 cup) plus 1 tablesp...


In [184]:
def preprocessing(text):
    # # Check if the text is a string
    if isinstance(text, str):
        # Apply string methods here
        # For example, converting to lower case
        return text.lower()
    else:
        # Return the text as is if it's not a string
        return text

data_content_based_filtering.Bahan = data_content_based_filtering.Bahan.apply(preprocessing)
data_content_based_filtering

,Food_Id,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bahan
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...",miso-butter roast chicken with acorn squash pa...
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...",crispy salt and pepper potatoes ['2 large egg ...
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...",thanksgiving mac and cheese ['1 cup evaporated...
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in...",italian sausage and bread stuffing ['1 (¾- to ...
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","newton's law ['1 teaspoon dark brown sugar', '..."
...,...,...,...,...,...,...,...
13496,13496,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"['1 cup all-purpose flour', '2/3 cup unsweeten...",brownie pudding cake ['1 cup all-purpose flour...
13497,13497,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",israeli couscous with roasted butternut squash...
13498,13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",rice-with-soy-glazed-bonito-flakes-and-sesame-...,['Leftover katsuo bushi (dried bonito flakes) ...,rice with soy-glazed bonito flakes and sesame ...
13499,13499,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,spanakopita-107344,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,spanakopita ['1 stick (1/2 cup) plus 1 tablesp...


In [185]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming tv is an instance of TfidfVectorizer or similar
tv = TfidfVectorizer()

# Replace NaN values with an empty string
data_content_based_filtering['Bahan'] = data_content_based_filtering['Bahan'].fillna('')

# Now apply fit_transform
vectors = tv.fit_transform(data_content_based_filtering['Bahan']).toarray()


In [186]:
similarity = cosine_similarity(vectors)
similarity[0][1:10]

array([0.1227871 , 0.19770426, 0.1112017 , 0.05992873, 0.09793242,
       0.142404  , 0.05774991, 0.17648535, 0.28990212])

In [187]:
def recommend_by_content_based_filtering(*bahan):
    # Convert bahan to lower case
    bahan = [item.lower() for item in bahan]

    # Check if any of the ingredients exist in the DataFrame
    if any(ingredient in sum(data_content_based_filtering['Cleaned_Ingredients'].apply(lambda x: x.lower().split()), []) for ingredient in bahan):
        # Find the index of the first occurrence of any of the ingredients
        bahan_index = data_content_based_filtering[data_content_based_filtering['Cleaned_Ingredients'].apply(lambda x: any(ingredient in x.lower() for ingredient in bahan))].index[0]
        distances = similarity[bahan_index]
        bahan_list = sorted(list(enumerate(distances)), key=lambda x: x[1], reverse=True)[1:10]

        recommended_bahan_makanan = []
        for i in bahan_list:
            recipe_data = data_content_based_filtering.iloc[i[0]]
            recommended_bahan_makanan.append({
                'Title': recipe_data.Title,
                'Ingredients': recipe_data.Cleaned_Ingredients,
                'Instructions': recipe_data.Instructions,
                'Similarity_Score': i[1]
            })

        return recommended_bahan_makanan
    else:
        # Handle the case where none of the ingredients are found
        return "No matching ingredients found in the dataset"


In [188]:
recommend_by_content_based_filtering('egg', 'potatoes')

[{'Title': 'Campfire Potatoes',
  'Ingredients': "['16 medium Yukon Gold potatoes (about 2 inches in diameter), halved', '3 tablespoons coarse kosher salt', '2 tablespoons extra-virgin olive oil', '4 unpeeled large garlic cloves, smashed', '1 teaspoon chopped fresh thyme', '1/2 teaspoon finely chopped fresh rosemary leaves']",
  'Instructions': 'Place potatoes in medium saucepan. Add enough cold water to cover by 1 inch. Sprinkle with 3 tablespoons coarse salt. Bring to boil, then reduce to simmer. Cook potatoes until tender, about 10 minutes. Drain. Return to saucepan. Add remaining ingredients to potatoes; toss to coat. Cool.\nArrange four 12-inch squares aluminum foil on work surface. Divide potato mixture among foil squares and place 1 garlic clove atop potatoes on each square. Crimp to seal. Store packets at room temperature up to 2 hours or in refrigerator up to 2 days.\nPrepare barbecue (medium-high heat). Place potato packets on grill and cook until heated through and sizzling,

#### Modeling with Collaborative Filtering

In [189]:
data_collaborative_filtering = data_food_loves.copy()
data_collaborative_filtering

,User_Id,Food_Id,Food_Loves
0,1,179,3
1,1,344,2
2,1,5,5
3,1,373,3
4,1,101,4
...,...,...,...
9995,300,425,2
9996,300,64,4
9997,300,311,3
9998,300,279,4


In [190]:
user_ids = data_collaborative_filtering['User_Id'].unique().tolist()
print('list userID: ', user_ids)

# Melakukan encoding userID
user_to_user_encoded = {x: i for i, x in enumerate(user_ids)}
print('encoded userID : ', user_to_user_encoded)

# Melakukan proses encoding angka ke ke userID
user_encoded_to_user = {i: x for i, x in enumerate(user_ids)}
print('encoded angka ke userID: ', user_encoded_to_user)

list userID:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219

In [191]:
favorite = data_collaborative_filtering['Food_Id'].unique().tolist()

# Melakukan proses encoding placeID
favorite_to_favorite_encoded = {x: i for i, x in enumerate(favorite)}

# Melakukan proses encoding angka ke placeID
favorite_encoded_to_favorite = {i: x for i, x in enumerate(favorite)}

In [193]:
# Mapping userID ke dataframe user
data_collaborative_filtering['user'] = data_collaborative_filtering['User_Id'].map(user_to_user_encoded)

# Mapping placeID ke dataframe resto
data_collaborative_filtering['favorite'] = data_collaborative_filtering['Food_Id'].map(favorite_to_favorite_encoded)

In [194]:
# Mendapatkan jumlah user
num_users = len(user_to_user_encoded)
print(num_users)

# Mendapatkan jumlah favorite
num_favorite = len(favorite_encoded_to_favorite)
print(num_favorite)

# Mengubah rating menjadi nilai float
data_collaborative_filtering['Food_Loves'] = data_collaborative_filtering['Food_Loves'].values.astype(np.float32)

# Nilai loves
sum_loves = (data_collaborative_filtering['Food_Loves'])



print('Number of User: {}, Number of Favorite: {}, Loves: {}'.format(
    num_users, num_favorite, sum_loves
))

300
437
Number of User: 300, Number of Favorite: 437, Loves: 0       3.0
1       2.0
2       5.0
3       3.0
4       4.0
       ... 
9995    2.0
9996    4.0
9997    3.0
9998    4.0
9999    2.0
Name: Food_Loves, Length: 10000, dtype: float32


In [195]:
data_collaborative_filtering = data_collaborative_filtering.sample(frac=1, random_state=42)
data_collaborative_filtering

,User_Id,Food_Id,Food_Loves,user,favorite
6252,188,207,3.0,187,262
4684,142,268,2.0,141,83
1731,54,103,3.0,53,10
4742,144,119,3.0,143,141
4521,138,288,5.0,137,283
...,...,...,...,...,...
5734,173,402,5.0,172,175
5191,157,85,4.0,156,33
5390,163,18,3.0,162,52
860,28,416,4.0,27,187


In [196]:
x = data_collaborative_filtering[['user', 'favorite']].values

# Membuat variabel y untuk membuat rating dari hasil
y = data_collaborative_filtering['Food_Loves'].values

# Membagi menjadi 80% data train dan 20% data validasi
train_indices = int(0.8 * data_collaborative_filtering.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:]
)

print(x, y)

[[187 262]
 [141  83]
 [ 53  10]
 ...
 [162  52]
 [ 27 187]
 [218   6]] [3. 2. 3. ... 3. 4. 2.]


In [197]:
num_users = 1000  # Replace with the actual number of users
num_favorite = 500  # Replace with the actual number of unique favorite items
embedding_size = 50  # This is a hyperparameter you can choose



class RecommenderNet(tf.keras.Model):
    def __init__(self, num_users, num_favorite, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_favorite = num_favorite
        self.embedding_size = embedding_size

        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer='he_normal',
            embeddings_regularizer=keras.regularizers.l2(1e-5)
        )
        self.user_bias = layers.Embedding(num_users, 1)

        self.favorite_embedding = layers.Embedding(
            num_favorite,
            embedding_size,
            embeddings_initializer='he_normal',
            embeddings_regularizer=keras.regularizers.l2(1e-5)
        )
        self.favorite_bias = layers.Embedding(num_favorite, 1)

        self.batch_norm = layers.BatchNormalization()
        self.dense = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        favorite_vector = self.favorite_embedding(inputs[:, 1])
        favorite_bias = self.favorite_bias(inputs[:, 1])

        dot_user_favorite = tf.tensordot(user_vector, favorite_vector, 2)

        x = dot_user_favorite + user_bias + favorite_bias
        x = self.batch_norm(x)
        x = self.dense(x)

        return x


In [198]:
# Model initialization
from tensorflow.keras.layers import Dropout

model = RecommenderNet(num_users, num_favorite, embedding_size)


# Model compilation
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)

# Example with dropout layers


In [199]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=8,
    epochs=100,
    verbose = 1, # You can increase epochs since early stopping will handle it.
    validation_data=(x_val, y_val),
    callbacks=[early_stopping]
)


Epoch 1/100
1000/1000 [==============================] - 7s 6ms/step - loss: 6.1972 - root_mean_squared_error: 2.4894 - val_loss: 6.1475 - val_root_mean_squared_error: 2.4794
Epoch 2/100
1000/1000 [==============================] - 5s 5ms/step - loss: 6.1813 - root_mean_squared_error: 2.4862 - val_loss: 6.1475 - val_root_mean_squared_error: 2.4794
Epoch 3/100
1000/1000 [==============================] - 4s 4ms/step - loss: 6.1813 - root_mean_squared_error: 2.4862 - val_loss: 6.1475 - val_root_mean_squared_error: 2.4794
Epoch 4/100
1000/1000 [==============================] - 5s 5ms/step - loss: 6.1813 - root_mean_squared_error: 2.4862 - val_loss: 6.1475 - val_root_mean_squared_error: 2.4794
Epoch 5/100
1000/1000 [==============================] - 5s 5ms/step - loss: 6.1813 - root_mean_squared_error: 2.4862 - val_loss: 6.1475 - val_root_mean_squared_error: 2.4794
Epoch 6/100
1000/1000 [==============================] - 4s 4ms/step - loss: 6.1813 - root_mean_squared_error: 2.4862 - val_l

In [204]:
favorite_df = data_food1
df = data_food_loves

# Mengambil sample user
user_id = df.User_Id.sample(1).iloc[0]
food_loves_by_user = df[df.User_Id == user_id] = df[df.User_Id == user_id]

In [205]:
food_not_loves = favorite_df[~favorite_df['Food_Id'].isin(food_loves_by_user.Food_Id.values)]['Food_Id']
food_not_loves = list(
    set(food_not_loves)
    .intersection(set(favorite_to_favorite_encoded.keys()))
)

food_not_loves = [[favorite_to_favorite_encoded.get(x)] for x in food_not_loves]
user_encoder = user_to_user_encoded.get(user_id)
user_favorite_array = np.hstack(
    ([[user_encoder]] * len(food_not_loves), food_not_loves)
)

In [210]:

    # If the array is not empty, proceed with prediction
    loves = model.predict(user_favorite_array).flatten()
    # Further code...


13/13 [==============================] - 0s 4ms/step


In [211]:
top_loves_indices = loves.argsort()[-10:][::-1]
recommended_favorite_ids = [
    favorite_encoded_to_favorite.get(food_not_loves[x][0]) for x in top_loves_indices
]

In [213]:

print('Recommendation for users: {}'.format(user_id))
print('===' * 9)
print('Your most like from user')
print('----' * 8)

top_food_user = (
    food_loves_by_user.sort_values(
        by = 'Food_Loves',
        ascending=False
    )
    .head(5)
    .Food_Id.values
)

favorite_df_rows = favorite_df[favorite_df['Food_Id'].isin(top_food_user)]
for row in favorite_df_rows.itertuples():
    print(row.Title)
    print('----' * 8)
print('Top 10 food recommendation')
print('----' * 8)


recommended_food = favorite_df[favorite_df['Food_Id'].isin(recommended_favorite_ids)]
for row in recommended_food.itertuples():
    print(row.Title)

Recommendation for users: 167
Your most like from user
--------------------------------
Crispy Salt and Pepper Potatoes
--------------------------------
Rosé All Day
--------------------------------
Syrniki (Сырники / Farmer’s Cheese Pancakes)
--------------------------------
Hawaij
--------------------------------
Spicy Citrus Refresher
--------------------------------
Top 10 food recommendation
--------------------------------
Salmon Burgers With Ginger and Quick-Pickled Cucumbers
Pork Meatballs and Cucumber Salad
Icebox Lemon Pie With Meringue
Frogmore Stew
Jalapeño Pepper Jelly
Spicy Sweet-and-Sour Salmon With Dates (Qaliyeh-e Khorma)
Chickpea and Chorizo Tostadas
Oaxaca Old Fashioned
East India Negroni
Watermelon, Lime, and White Pepper Yogurt Drink


In [214]:
import tensorflow as tf

model = tf.keras.models.load_model('recommendation_recepify')


In [215]:
converter = tf.lite.TFLiteConverter.from_saved_model('recommendation_recepify')


In [216]:
tflite_model = converter.convert()


In [217]:
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [218]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
